In [ ]:
!wget https://github.com/lucidfrontier45/PyTorch-Book/blob/master/data/taco_and_burrito.tar.gz

--2021-01-16 10:08:06--  https://github.com/lucidfrontier45/PyTorch-Book/blob/master/data/taco_and_burrito.tar.gz
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘taco_and_burrito.tar.gz’

taco_and_burrito.ta     [ <=>                ]  85.03K  --.-KB/s    in 0.05s   

2021-01-16 10:08:06 (1.57 MB/s) - ‘taco_and_burrito.tar.gz’ saved [87070]



In [17]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

from torchvision.datasets import FashionMNIST
from torchvision import transforms

In [19]:
# データ読み込み
fashion_mnist_train = FashionMNIST(root="./FashionMNIST",
                                   train=True,
                                   download=True, 
                                   transform=transforms.ToTensor())
fashion_mnist_test = FashionMNIST(root="./FashionMNIST",
                                   train=False,
                                   download=True,
                                   transform=transforms.ToTensor())

batch_size = 128
train_loader = DataLoader(fashion_mnist_train,
                           batch_size=batch_size, shuffle=True)
test_loader = DataLoader(fashion_mnist_test,
                          batch_size=batch_size, shuffle=False)

In [21]:
# CNNの定義

conv_net = nn.Sequential(
    nn.Conv2d(1, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    nn.Flatten()
)

# データを入力した時の, 出力サイズを確認
test_input = torch.ones(1, 1, 28, 28)
conv_output_size = conv_net(test_input).size()[-1]

In [22]:
conv_output_size

1024

In [24]:
mlp = nn.Sequential(
    nn.Linear(conv_output_size, 200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout2d(0.25),
    nn.Linear(200, 10)
)

In [26]:
net = nn.Sequential(
    conv_net,
    mlp
)
net

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout2d(p=0.25, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): Flatten(start_dim=1, end_dim=-1)
  )
  (1): Sequential(
    (0): Linear(in_features=1024, out_features=200, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout2d(p=0.25, inplace=False)
    (4): Linear(in_features=200, out_features=10, bias=True)
  )
)

In [27]:
# 評価のヘルパー関数を定義

def eval_net(net, data_loader, device='cuda:0'):
  net.eval()
  ys = []
  ypred = []

  for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)

    with torch.no_grad():
      _, y_pred = net(x).max(1)

    ys.append(y)
    ypreds.append(y_pred)

    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)

    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()